In [1]:
pip install statsmodels

     |████████████████████████████████| 10.8MB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 38.6MB 77.3MB/s eta 0:00:01
     |████████████████████████████████| 235kB 91.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install fosforml

     |████████████████████████████████| 51kB 4.9MB/s eta 0:00:011
     |████████████████████████████████| 10.9MB 13.2MB/s eta 0:00:01
     |████████████████████████████████| 1.9MB 65.7MB/s eta 0:00:01
     |████████████████████████████████| 307kB 112.5MB/s eta 0:00:01
     |████████████████████████████████| 200.3MB 87kB/s  eta 0:00:01
     |████████████████████████████████| 51kB 17.1MB/s eta 0:00:01
     |████████████████████████████████| 1.3MB 105.4MB/s eta 0:00:01
     |████████████████████████████████| 81kB 31.9MB/s  eta 0:00:01
     |████████████████████████████████| 98.7MB 116kB/s eta 0:00:0101��███████████████▉            | 61.1MB 1.6MB/s eta 0:00:24
     |████████████████████████████████| 133kB 89.4MB/s eta 0:00:01
     |████████████████████████████████| 39.9MB 50.5MB/s eta 0:00:01
     |████████████████████████████████| 174kB 72.2MB/s eta 0:00:01
     |████████████████████████████████| 2.5MB 75.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 21.7MB/s eta 0:00:01


In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()
 
table_name = '"FACT_CUSTOMER_ORDERS"'
 
sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [3]:
df.columns

Index(['ORDER_ID', 'PRODUCT_ID', 'ORDER_QTY', 'ORDER_DATE'], dtype='object')

In [4]:
data = df[['PRODUCT_ID','ORDER_DATE','ORDER_QTY']]

In [5]:
data['ORDER_DATE']=pd.to_datetime(data['ORDER_DATE'])

In [6]:
data_grouped = data.groupby(['PRODUCT_ID',data['ORDER_DATE'].dt.to_period('M')]).agg(Total_Order_Qty=('ORDER_QTY','sum')).reset_index()

In [7]:
New_data_grouped= data.groupby(['PRODUCT_ID',data['ORDER_DATE'].dt.to_period('M')]).agg(Total_Order_Qty=('ORDER_QTY','sum')).reset_index()

In [8]:
New_data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2992 entries, 0 to 2991
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype    
---  ------           --------------  -----    
 0   PRODUCT_ID       2992 non-null   object   
 1   ORDER_DATE       2992 non-null   period[M]
 2   Total_Order_Qty  2992 non-null   int64    
dtypes: int64(1), object(1), period[M](1)
memory usage: 70.2+ KB


In [9]:
data_grouped['ORDER_MONTH']=data_grouped['ORDER_DATE'].dt.to_timestamp()

In [10]:
New_data_grouped['ORDER_MONTH']=New_data_grouped['ORDER_DATE'].dt.to_timestamp()

In [11]:
New_data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2992 entries, 0 to 2991
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PRODUCT_ID       2992 non-null   object        
 1   ORDER_DATE       2992 non-null   period[M]     
 2   Total_Order_Qty  2992 non-null   int64         
 3   ORDER_MONTH      2992 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1), period[M](1)
memory usage: 93.6+ KB


In [12]:
data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2992 entries, 0 to 2991
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PRODUCT_ID       2992 non-null   object        
 1   ORDER_DATE       2992 non-null   period[M]     
 2   Total_Order_Qty  2992 non-null   int64         
 3   ORDER_MONTH      2992 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1), period[M](1)
memory usage: 93.6+ KB


In [13]:
data_grouped = data_grouped[['PRODUCT_ID','ORDER_MONTH','Total_Order_Qty']]

In [14]:
New_data_grouped= data_grouped[['PRODUCT_ID','ORDER_MONTH','Total_Order_Qty']]

In [15]:
data_grouped

,PRODUCT_ID,ORDER_MONTH,Total_Order_Qty
0,Product_001,2019-01-01,148400
1,Product_001,2019-02-01,146650
2,Product_001,2019-03-01,133150
3,Product_001,2019-04-01,153600
4,Product_001,2019-05-01,154650
...,...,...,...
2987,Product_100,2024-04-01,30200
2988,Product_100,2024-05-01,27850
2989,Product_100,2024-06-01,33200
2990,Product_100,2024-07-01,33300


In [16]:
data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2992 entries, 0 to 2991
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   PRODUCT_ID       2992 non-null   object        
 1   ORDER_MONTH      2992 non-null   datetime64[ns]
 2   Total_Order_Qty  2992 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 70.2+ KB


In [17]:
data_grouped = data_grouped.set_index('ORDER_MONTH')

In [18]:
data_grouped.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2992 entries, 2019-01-01 to 2024-08-01
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PRODUCT_ID       2992 non-null   object
 1   Total_Order_Qty  2992 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 70.1+ KB


In [19]:
ORD_grouped_data = data_grouped.groupby('ORDER_MONTH')

In [20]:
print(ORD_grouped_data)

In [21]:
# Group data by 'product_id'
PID_grouped_data = data_grouped.groupby('PRODUCT_ID')

In [22]:
# Initialize an empty DataFrame for forecast results
test_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_MONTH', 'test_forecast_orders','rse', 'rmse','r2_score'])
test_results.set_index('ORDER_MONTH')

,PRODUCT_ID,test_forecast_orders,rse,rmse,r2_score
ORDER_MONTH,,,,,


In [23]:
# Initialize an empty DataFrame for forecast results
forecast_results = pd.DataFrame(columns=['PRODUCT_ID','ORDER_MONTH', 'forecast_order_qty'])
forecast_results.set_index('ORDER_MONTH')

,PRODUCT_ID,forecast_order_qty
ORDER_MONTH,,


In [24]:
print(PID_grouped_data)

In [25]:
for product_id, group in PID_grouped_data:
    # Sort data by order date
    group.sort_index(inplace=True)
    
    # Split data into train and test sets (80% train, 20% test)
    train_size = int(0.8 * len(group))
    train_data, test_data = group.iloc[:train_size], group.iloc[train_size:]

    
    
    # Create SARIMAX model
    model = SARIMAX(train_data['Total_Order_Qty'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
    model_fit = model.fit(disp=False)
    
    # Forecast the next period
    test_model = model_fit.get_forecast(steps=len(test_data))
    test_df = test_model.predicted_mean

    #print(f'test_df:{test_df.index}')
    
    # Calculate RMSE for order quantity
    #rmse_order_qty = np.sqrt(mean_squared_error(test_data['ORDER_QTY'], forecast_order_qty))
    
    # Evaluate the model
    residuals=test_df.tolist()-test_data['Total_Order_Qty']
    rse = np.sqrt(((residuals) ** 2).sum().sum() / (test_data.shape[0] * test_data.shape[1] - len(model_fit.params)))
    rmse = np.sqrt(mean_squared_error(test_data['Total_Order_Qty'], test_df))
    r2 = r2_score(test_data['Total_Order_Qty'], test_df)

    
    date_list = test_df.index.to_list()
    #date_list = train_data.index.to_list()

    
    #print(f"Order_date: {date_list}")    
    new_row = {
        'PRODUCT_ID': product_id,
        'ORDER_MONTH': date_list,
        'test_forecast_orders': test_df,
        'rse': rse,
        'rmse': rmse,
        'r2_score': r2
    }
    
    df_new = pd.DataFrame(new_row)

    df_new.set_index('ORDER_MONTH')
    
    test_results=pd.concat([test_results,df_new],ignore_index=True)
    print(f'{test_results}')
    # Forecast future values
    forecast = model_fit.get_forecast(steps=25)#, steps=n_forecast)
    forecast_df = forecast.predicted_mean
    
    date_list2= forecast_df.index.to_list()
    
    new_row2 = {
        'product_id': product_id,
        'order_date': date_list2,
        'forecast_order_qty': forecast_df,
    }
    
    df_new2 = pd.DataFrame(new_row2)
    
    forecast_results=pd.concat([forecast_results,df_new2],ignore_index=True)

     PRODUCT_ID ORDER_MONTH  test_forecast_orders           rse          rmse  \
0   Product_001  2023-07-01         118823.028009  12247.075647  15697.565379   
1   Product_001  2023-08-01         101123.262760  12247.075647  15697.565379   
2   Product_001  2023-09-01         105282.149996  12247.075647  15697.565379   
3   Product_001  2023-10-01         117745.845182  12247.075647  15697.565379   
4   Product_001  2023-11-01         110611.593083  12247.075647  15697.565379   
5   Product_001  2023-12-01         109442.188136  12247.075647  15697.565379   
6   Product_001  2024-01-01         123789.859016  12247.075647  15697.565379   
7   Product_001  2024-02-01         121702.673686  12247.075647  15697.565379   
8   Product_001  2024-03-01         126125.919391  12247.075647  15697.565379   
9   Product_001  2024-04-01         122617.484315  12247.075647  15697.565379   
10  Product_001  2024-05-01         113678.212178  12247.075647  15697.565379   
11  Product_001  2024-06-01 